# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', con=engine)
X = df['message']
Y = df.drop(['message', 'id', 'original', 'genre'], axis=1)

In [3]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """
    Function to enable text tokenization
    
    """
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train the pipeline
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# Make predictions on the test set
Y_pred = pipeline.predict(X_test)

# To get f1 score, precision, and recall for each output category
def test_model(y_test, y_pred): 
    """
    Function to iterate through columns and call sklearn classification report on each.
    """
    for i, column in enumerate(Y_test):
        print(f"Category: {column}")
        print(classification_report(Y_test[column], Y_pred[:, i]))

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# Define the parameters for grid search
pipeline.get_params()

{'memory': None,
 'steps': [('tfidf',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f4ae2259ea0>, use_idf=True,
           vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,

In [9]:
# specify parameters for grid search 
parameters = {
    'clf__estimator__n_estimators': [50, 100]
}

In [10]:
# Perform grid search with cross-validation
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs=-1)

cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [11]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=50 .................................
[CV] clf__estimator__n_estimators=100 ................................
[CV] clf__estimator__n_estimators=100 ................................
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=50, score=0.23272779287655557, total=19.0min
[CV]  clf__estimator__n_estimators=50, score=0.2329041487839771, total=19.3min


[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed: 21.3min remaining: 42.7min


[CV]  clf__estimator__n_estimators=50, score=0.23172650550708054, total=19.4min
[CV]  clf__estimator__n_estimators=100, score=0.2318695465598627, total=28.6min
[CV]  clf__estimator__n_estimators=100, score=0.23476394849785406, total=28.7min
[CV]  clf__estimator__n_estimators=100, score=0.23387212129881277, total=28.7min


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 30.3min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [12]:
# Print the best parameters found by grid search
print("Best Parameters:", cv.best_params_)

Best Parameters: {'clf__estimator__n_estimators': 100}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
# Make predictions on the test set using the tuned model
y_pred = cv.predict(X_test)

In [14]:
test_model(Y_test, y_pred)

Category: related
             precision    recall  f1-score   support

          0       0.61      0.35      0.44      1266
          1       0.81      0.93      0.87      3938
          2       1.00      0.10      0.18        40

avg / total       0.76      0.78      0.76      5244

Category: request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      4349
          1       0.80      0.34      0.48       895

avg / total       0.87      0.87      0.85      5244

Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

Category: aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3113
          1       0.74      0.50      0.60      2131

avg / total       0.73      0.73      0.71      5244

Category: med

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
# To check accuracy, precision, and recall of the tuned model
accuracy = (y_pred == Y_test).mean()
accuracy

related                   0.793478
request                   0.891686
offer                     0.995042
aid_related               0.776697
medical_help              0.922960
medical_products          0.949466
search_and_rescue         0.977689
security                  0.983028
military                  0.970633
child_alone               1.000000
water                     0.945080
food                      0.920862
shelter                   0.927155
clothing                  0.986651
money                     0.981503
missing_people            0.988749
refugees                  0.967010
death                     0.958429
other_aid                 0.869565
infrastructure_related    0.937452
transport                 0.956522
buildings                 0.951182
electricity               0.977117
tools                     0.993898
hospitals                 0.991228
shops                     0.995805
aid_centers               0.987223
other_infrastructure      0.957475
weather_related     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [16]:
import pickle
filename = 'model.pkl'
pickle.dump(cv_estimator, open(filename, 'wb'))

NameError: name 'cv_estimator' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.